In [124]:
import pandas as pd
df= pd.read_csv('./j_detail.csv')


import string
import nltk
import re
import jieba
ps = nltk.PorterStemmer()
stopwords =[]
with open("./stopwords.txt") as file:  
    stopwords = file.read() 

# stopwords =nltk.corpus.stopwords.words('english')
all_stop_words = stopwords.split('\n')



def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

def jiebaclearText(text, all_stop_words=''):
    
    mywordlist = ''
    seg_list = jieba.cut(text, cut_all=False, HMM=True)
    f_stop_seg_list=all_stop_words
    for myword in seg_list:
        if not(myword.strip() in f_stop_seg_list) and len(myword.strip())>1:
            mywordlist = mywordlist + ' ' + myword
    return mywordlist
df['clean'] = df['jd'].apply(lambda x: jiebaclearText(x))


words = ''
for w in df['clean']:
    words = words + ' '+ w
words = words.split()



In [135]:
#in fact need a set

from nltk.probability import FreqDist
fdist = FreqDist(words)
tops=fdist.most_common(1000)


# import collections
# collections.Counter(words)

#keep skillset only

In [160]:
r1=tops[-10:][1]
print(r1[0])
pattern=re.compile("[^A-Za-z]")
a=pattern.match( r1[0])
re.findall("[^A-Za-z]",r1[0])

tensorflow


[]

In [162]:
from nltk.corpus import words
"技术" in words.words()

False